## Launch the STEMMUS_SCOPE model
Steps to run the STEMMUS_SCOPE model, including preprocessing and postprocessing, on Surf super computer Snellius.

In [ ]:
import os
import shutil
import numpy as np
from pathlib import Path
import subprocess
import datetime
import time

#### Create an executable file of STEMMUS_SCOPE

In [ ]:
# path to executable
root_path_to_model = Path(Path.home(),"EcoExtreML", "STEMMUS_SCOPE")
# submit slurm job
result = subprocess.run(["sbatch", Path(root_path_to_model, "exe", "build_stemmus_scope_exe.sh")])
result.check_returncode()

#### Update/set config files

In [ ]:
# the user must provide the correct path
path_dict = {
    "SoilPropertyPath": "/data/shared/EcoExtreML/STEMMUS_SCOPEv1.0.0/input/SoilProperty/",
    "InputPath": "/data/shared/EcoExtreML/STEMMUS_SCOPEv1.0.0/input/",
    "OutputPath": "/data/shared/EcoExtreML/STEMMUS_SCOPEv1.0.0/output/",
    "ForcingPath": "/data/shared/EcoExtreML/STEMMUS_SCOPEv1.0.0/input/Plumber2_data/",
    "ForcingFileName": "AU-DaS_2010-2017_OzFlux_Met.nc",
    "VegetationPropertyPath": "/data/shared/EcoExtreML/STEMMUS_SCOPEv1.0.0/model_parameters/vegetation_property/",
    "DurationSize": "NA"
}

# generate a text file
root_path = Path().resolve().parent

with open(Path(root_path, "config_file_snellius.txt"), 'w') as f:
    print("The listed paths are defined in the config file:")
    for i in path_dict.keys():
        print(i)
        f.write((i + "=" + path_dict[i] + "\n"))

#### Create input directories, prepare input files 

In [ ]:
def input_dir(station_name):
    """Create input directory and prepare input files
    """
    # get start time with the format Y-M-D-HM
    start_time = str(datetime.datetime.now()).split('.')[0]
    timestamp = start_time.replace(' ', '-').replace(':', '')[:-2]
    # create input directory
    work_dir = Path(path_dict["InputPath"], station_name + '_' + timestamp)
    Path(work_dir).mkdir(parents=True, exist_ok=True)
    print(f"Prepare work directory for the station: {station_name}")
    # copy model parameters to work directory
    shutil.copytree(path_dict["VegetationPropertyPath"], work_dir, dirs_exist_ok=True)
    # update config file for ForcingFileName and InputPath
    with open(Path(work_dir, f"{station_name}_{timestamp}_config.txt"), 'w') as f:
        for i in path_dict.keys():
            if i == "ForcingFileName":
                f.write(i + "=" + ncfile.stem + ".nc" + "\n")
            elif i == "InputPath":
                f.write(i + "=" + str(work_dir) + "\n")
            else:
                f.write(i + "=" + path_dict[i] + "\n")
    
    # create an output file
    Path(root_path, ".out").mkdir(parents=True, exist_ok=True)
    with open(Path(root_path, ".out", f"{station_name}.out"), 'w') as f:
        for i in path_dict.keys():
            if i == "ForcingFileName":
                f.write(i + " is " + ncfile.stem + ".nc" + "\n")
            elif i == "InputPath":
                f.write(i + " is " + str(work_dir) + "\n")
            elif i == "DurationSize":
                pass
            else:
                f.write(i + " is " + path_dict[i] + "\n")
    

In [ ]:
# specify the station of interest
# station_names_list is by default none, if it is not specified, then all stations with forcing
# listed in the ForcingPath will be used.
station_names_list = None
#station_names_list = ["US-Myb", "ZA-Kru"]

for ncfile in Path(path_dict["ForcingPath"]).iterdir():
    # get the station name
    station_name = ncfile.stem.split('_')[0]
    # check if a station list is provided
    if station_names_list == None:
        input_dir(station_name)
    elif station_name in station_names_list:
        input_dir(station_name)

#### Run the model

In [ ]:
for station_dir in os.listdir(path_dict["InputPath"]):
    station_name = station_dir.split("_")[0]
    path_to_config = Path(path_dict["InputPath"], f"{station_dir}",f"{station_dir}_config.txt")
    # set matlab log dir to slurm, otherwise java.log files are created in user home dir
    subprocess.run(["export", "MATLAB_LOG_DIR=../out"])
    # start time
    start_time = time.time()
    # run the model
    result = subprocess.run(["exe/STEMMUS_SCOPE", path_to_config, ">>", f"{station_name}.out"])
    result.check_returncode()
    run_time = time.time() - start_time
    # add execution information to .out file
    with open(Path(root_path, ".out", f"{station_name}.out"), 'a') as f:
        f.write(f"Run is COMPLETED. Model run time is {run_time} s.")

#### Create output directories, prepare output files

In [ ]:
# convert csv files to nc files
path_to_utils = Path(root_path_to_model, "utils/csv_to_nc")
for station_dir in os.listdir(path_dict["InputPath"]):
    path_to_config = Path(path_dict["InputPath"], f"{station_dir}",f"{station_dir}_config.txt")
    result = subprocess.run(["python", Path(path_to_utils, "generate_netcdf_files.py"),
         "--config_file", path_to_config, "--variable_file", Path(path_to_utils, "Variables_will_be_in_NetCDF_file.csv")])
    result.check_returncode()